In [29]:
import keras

from keras.models import Model
from keras.models import Sequential
from keras.models import load_model

from keras.layers import Activation,BatchNormalization
from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate, Lambda
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM,SimpleRNN

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical

import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 导入自定义库
from utils.data_utils import clean_str
from utils.data_utils import build_vocab
from utils.data_utils import get_tokens

In [3]:
# 加载训练集合
df_train_dataset = pd.read_csv('./data/training-inspur.csv', encoding='utf-8')
# 加载测试集
df_test_dataset = pd.read_csv('./data/Preliminary-texting-1.csv', encoding='utf-8')
# 查看数据集合 shape
df_test_dataset.shape

(102024, 2)

In [4]:
# 提取数据集中所需的字段
df_train_dataset = df_train_dataset[['COMMCONTENT', 'COMMLEVEL']]
df_test_dataset = df_test_dataset[['COMMCONTENT']]

In [5]:
# 合并数据集用于构建词汇表
df_all_dataset = pd.concat([df_train_dataset, df_test_dataset], ignore_index=True)
df_all_dataset.shape

(122024, 2)

In [6]:
BASE_DIR = '/Users/tsw/ScenicSpotReviews'

W2V_DIR = BASE_DIR + '/embeddings/'

TEXT_DATA_DIR = BASE_DIR + '/data/'

MAX_NUM_WORDS = 33950

MAX_SEQUENCE_LENGTH = 150 # 每篇文章选取150个词

MAX_NB_WORDS = 80000 # 将字典设置为含有1万个词84480

EMBEDDING_DIM = 300 # 词向量维度，300维

VALIDATION_SPLIT = 0.1 # 测试集大小，全部数据的10%

BATCH_SIZE = 128

NUM_LABELS = 3

In [7]:
def seg_corpus(corpus):
    seg_corpus = []
    for line in corpus:
        line = str(line).strip()
        seg_list = jieba.cut(line, cut_all=False)
        # 过滤空字符
        seg_list = [w for w in seg_list if w != ' ']
        seg_corpus.append(" ".join(seg_list))
    return seg_corpus

In [8]:
# 对所有文本分词
seged_text = seg_corpus(df_all_dataset['COMMCONTENT'])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/5n/2_by50851fxc4d_snc1d9wf80000gn/T/jieba.cache
Loading model cost 0.859 seconds.
Prefix dict has been built succesfully.


In [9]:
# 将分词后的数据并入 df_all_dataset
df_all_dataset['COMMCONTENT_SEG'] = pd.DataFrame(seged_text,columns=['COMMCONTENT_SEG'])
df_all_dataset.head()

,COMMCONTENT,COMMLEVEL,COMMCONTENT_SEG
0,普通公园一个只是多了几个泉而已，人不多，适合老人孩子闲逛，买票的话还是贵了，人家说6.30之...,1.0,普通 公园 一个 只是 多 了 几个 泉 而已 ， 人不多 ， 适合 老人 孩子 闲逛 ， ...
1,跟儿子在里面玩了一天，非常好！跟儿子在里面玩了一天，非常好！真的很不错哦，有空还要去,1.0,跟 儿子 在 里面 玩 了 一天 ， 非常 好 ！ 跟 儿子 在 里面 玩 了 一天 ， 非...
2,这已经是第五次来这里玩了。每次孩子都很喜欢，不愿意从水里出来。有机会还会再来。还有比我更忠诚...,1.0,这 已经 是 第五次 来 这里 玩 了 。 每次 孩子 都 很 喜欢 ， 不 愿意 从水里 ...
3,当天在携程上定的票，打温泉度假村咨询电话和携程客服都说次日生效，但到酒店后，票能用。请客服人...,1.0,当天 在 携程 上定 的 票 ， 打 温泉 度假村 咨询电话 和 携程 客服 都 说 次日 ...
4,烟台历史的一部分，非常值得推荐去看看！海边景色也很漂亮！,1.0,烟台 历史 的 一部分 ， 非常 值得 推荐 去 看看 ！ 海边 景色 也 很漂亮 ！


In [10]:
# text_corpus
text_corpus = df_all_dataset['COMMCONTENT_SEG']
# 传入我们词向量的字典
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) 
# 传入我们的训练数据，得到训练数据中出现的词的字典
tokenizer.fit_on_texts(text_corpus) 

In [11]:
# 根据训练数据中出现的词的字典，将训练数据转换为sequences
dataset_sequences = tokenizer.texts_to_sequences(text_corpus) 

In [12]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 100134 unique tokens.


In [13]:
padded_dataset_sequences = pad_sequences(dataset_sequences, maxlen=MAX_SEQUENCE_LENGTH)
padded_dataset_sequences.shape

(122024, 150)

In [14]:
df_test_dataset_seg = df_all_dataset['COMMCONTENT_SEG'][20000:]
test_dataset_sequences = tokenizer.texts_to_sequences(df_test_dataset_seg)
padded_test_dataset_sequences = pad_sequences(test_dataset_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [15]:
# 划分训练集和测试集
from sklearn.model_selection import train_test_split
train_X,valid_X,train_y,valid_y =train_test_split(padded_dataset_sequences[:df_train_dataset.shape[0]], 
                                                  df_all_dataset['COMMLEVEL'][:df_train_dataset.shape[0]], 
                                                  test_size=0.1)

In [16]:
# label one-hot 表示
labels = df_all_dataset['COMMLEVEL'].dropna().map(int)#.values.tolist()
labels = to_categorical(labels-1) 



In [17]:
vocab,vocab_freqs = build_vocab(df_all_dataset['COMMCONTENT_SEG'])
vocab_size = min(MAX_NB_WORDS, len(vocab_freqs)) + 2
word2index = {x[0]: i+2 for i, x in enumerate(vocab_freqs.most_common(MAX_NB_WORDS))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}
len(word2index)

80002

In [18]:
print('Indexing word embeddings.')  
embeddings_index = {}
with open('./embeddings/sgns.weibo.word','r') as f:
    f = f.readlines()
    for i in f[1:]:
        values = i.strip().split(' ')
        word = str(values[0])
        embedding = np.asarray(values[1:],dtype='float')
        embeddings_index[word] = embedding
print('word embedding',len(embeddings_index))

Indexing word embeddings.
word embedding 195201


In [19]:
nb_words = min(MAX_NB_WORDS,len(word2index))
nb_words

80000

In [20]:
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

In [21]:
for word, i in word2index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(str(word).upper())
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

In [ ]:
pre_embedding_layer =  Embedding(input_dim = nb_words+1, 
                             output_dim = EMBEDDING_DIM, 
                            weights=[word_embedding_matrix], 
                             input_length=MAX_SEQUENCE_LENGTH, 
                             mask_zero=True,
                             trainable=False
                            )

In [26]:
from keras.layers import Lambda
from keras import backend as K

In [67]:
def model_conv1D_(emb_matrix, max_len):
    
    # The embedding layer containing the word vectors
    emb_layer = Embedding(
        input_dim=emb_matrix.shape[0],
        output_dim=emb_matrix.shape[1],
        weights=[emb_matrix],
        input_length=max_len,
        trainable=True
    )
    
    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    conv2 = Conv1D(filters=128, kernel_size=2, padding='same', activation='relu')
    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    conv4 = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')
    conv5 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')
    conv6 = Conv1D(filters=32, kernel_size=6, padding='same', activation='relu')

    # Define inputs
    seq1 = Input(shape=(max_len,))
#     seq2 = Input(shape=(max_len,))

    # Run inputs through embedding
    emb1 = emb_layer(seq1)
#     emb2 = emb_layer(seq2)
    emb1 = SpatialDropout1D(0.3)(emb1)
    emb1 = Bidirectional(GRU(100, return_sequences=True))(emb1)

    # Run through CONV + GAP layers
    conv1a = conv1(emb1)
    glob1a = GlobalAveragePooling1D()(conv1a)
#     conv1b = conv1(emb2)
#     glob1b = GlobalAveragePooling1D()(conv1b)

    conv2a = conv2(emb1)
    glob2a = GlobalAveragePooling1D()(conv2a)
#     conv2b = conv2(emb2)
#     glob2b = GlobalAveragePooling1D()(conv2b)

    conv3a = conv3(emb1)
    glob3a = GlobalAveragePooling1D()(conv3a)
#     conv3b = conv3(emb2)
#     glob3b = GlobalAveragePooling1D()(conv3b)

    conv4a = conv4(emb1)
    glob4a = GlobalAveragePooling1D()(conv4a)
#     conv4b = conv4(emb2)
#     glob4b = GlobalAveragePooling1D()(conv4b)

    conv5a = conv5(emb1)
    glob5a = GlobalAveragePooling1D()(conv5a)
#     conv5b = conv5(emb2)
#     glob5b = GlobalAveragePooling1D()(conv5b)

    conv6a = conv6(emb1)
    glob6a = GlobalAveragePooling1D()(conv6a)
#     conv6b = conv6(emb2)
#     glob6b = GlobalAveragePooling1D()(conv6b)

    mergea = concatenate([glob1a, glob2a, glob3a, glob4a, glob5a, glob6a])
#     mergeb = concatenate([glob1b, glob2b, glob3b, glob4b, glob5b, glob6b])

    # We take the explicit absolute difference between the two sentences
    # Furthermore we take the multiply different entries to get a different measure of equalness
#     diff = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(4 * 128 + 2*32,))([mergea, mergeb])
#     mul = Lambda(lambda x: x[0] * x[1], output_shape=(4 * 128 + 2*32,))([mergea, mergeb])
    
#     diff = Lambda(lambda x: K.max(x), output_shape=(64,))(mergea)
#     mul = Lambda(lambda x: x[0] * x[1], output_shape=(4 * 128 + 2*32,))([mergea])

    # Add the magic features
#     magic_input = Input(shape=(5,))
#     magic_dense = BatchNormalization()(magic_input)
#     magic_dense = Dense(64, activation='relu')(magic_dense)

    # Add the distance features (these are now TFIDF (character and word), Fuzzy matching, 
    # nb char 1 and 2, word mover distance and skew/kurtosis of the sentence vector)
#     distance_input = Input(shape=(20,))
#     distance_dense = BatchNormalization()(distance_input)
#     distance_dense = Dense(128, activation='relu')(distance_dense)

    # Merge the Magic and distance features with the difference layer
#     merge = concatenate([diff, mul, magic_dense, distance_dense])
#     merge = concatenate([diff, mul])

    # The MLP that determines the outcome
    x = Dropout(0.2)(mergea)
#     x = BatchNormalization()(x)
    x = Dense(300, activation='relu')(x)

#     x = Dropout(0.2)(x)
#     x = BatchNormalization()(x)
    pred = Dense(3, activation='softmax')(x)

    # model = Model(inputs=[seq1, seq2, magic_input, distance_input], outputs=pred)
#     model = Model(inputs=[seq1, seq2, magic_input, distance_input], outputs=pred)
    model = Model(inputs=seq1, outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [68]:
model = model_conv1D_(word_embedding_matrix,MAX_SEQUENCE_LENGTH)

In [69]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 150, 300)     24000300    input_22[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 150, 300)     0           embedding_17[0][0]               
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 150, 200)     240600      spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
conv1d_97 

In [70]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/5
18000/18000 [==============================] - 242s 13ms/step - loss: 1.0613 - acc: 0.4230 - val_loss: 0.8996 - val_acc: 0.5765
Epoch 2/5
18000/18000 [==============================] - 224s 12ms/step - loss: 0.8330 - acc: 0.6076 - val_loss: 0.7803 - val_acc: 0.6460
Epoch 3/5
18000/18000 [==============================] - 222s 12ms/step - loss: 0.7190 - acc: 0.6742 - val_loss: 0.7369 - val_acc: 0.6600
Epoch 4/5
18000/18000 [==============================] - 223s 12ms/step - loss: 0.6280 - acc: 0.7291 - val_loss: 0.7648 - val_acc: 0.6635
Epoch 5/5
18000/18000 [==============================] - 220s 12ms/step - loss: 0.5308 - acc: 0.7839 - val_loss: 0.7685 - val_acc: 0.6710


In [71]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 222s 12ms/step - loss: 0.4478 - acc: 0.8255 - val_loss: 0.8174 - val_acc: 0.6640


In [72]:
all_test_preds = model.predict(padded_test_dataset_sequences, batch_size=1024)
conv1d = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(conv1d).value_counts(normalize=True)

2    0.377911
1    0.332098
3    0.289991
dtype: float64

In [73]:
np.savetxt("lstm+conv1d-0.6640-0.33-0.37-0.28.txt", conv1d,fmt="%d")

In [58]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 104s 6ms/step - loss: 0.8070 - acc: 0.6355 - val_loss: 0.8080 - val_acc: 0.6255


In [59]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 99s 6ms/step - loss: 0.7753 - acc: 0.6542 - val_loss: 0.8023 - val_acc: 0.6280


In [60]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 101s 6ms/step - loss: 0.7448 - acc: 0.6731 - val_loss: 0.7807 - val_acc: 0.6420


In [61]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 103s 6ms/step - loss: 0.7223 - acc: 0.6852 - val_loss: 0.7771 - val_acc: 0.6420


In [62]:
all_test_preds = model.predict(padded_test_dataset_sequences, batch_size=1024)
conv1d = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(conv1d).value_counts(normalize=True)

2    0.375216
1    0.354554
3    0.270231
dtype: float64

In [ ]:
np.savetxt("lstm+conv1d-0.6770-result.txt", conv1d,fmt="%d")

In [ ]:
def get_rnn_cnn_model():
    embedding_dim = 300
    inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    x =  Embedding(input_dim = nb_words+1, 
                             output_dim = EMBEDDING_DIM, 
                             weights=[word_embedding_matrix], 
                             input_length=MAX_SEQUENCE_LENGTH, 
                             mask_zero=False,
                             trainable=True
                            )(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    kmax_pool = Lambda(lambda x: K.max(x, axis=1), output_shape=(64,))(x)
    conc = concatenate([avg_pool, max_pool, kmax_pool])
    outp = Dense(3, activation="softmax")(conc)

    model = Model(inputs=inp, outputs=outp)
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [ ]:
rnn_cnn_model = get_rnn_cnn_model()

plot_model(rnn_cnn_model, to_file='./rnn_cnn_model.png', 
show_shapes=True, show_layer_names=True)

!['./rnn_cnn_model.png'](./rnn_cnn_model.png)

In [ ]:
batch_size = 256
epochs = 4

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

In [ ]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=1024)
w2v7 = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(w2v7).value_counts(normalize=True)

In [ ]:
# tainable = True
batch_size = 256
epochs = 4

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

In [ ]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=1024)
w2v8 = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(w2v8).value_counts(normalize=True)

In [ ]:
# add kmax pooling

In [ ]:
batch_size = 256
epochs = 4

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

In [ ]:
batch_size = 256
epochs = 1

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

In [ ]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=256)
rnn_cnn_k_max = np.argmax(all_test_preds,axis=1)[:]+1

In [ ]:
pd.Series(rnn_cnn_k_max).value_counts(normalize=True)

In [ ]:
np.savetxt("rnn-cnn-0.6685-result.txt", rnn_cnn_k_max,fmt="%d")

In [ ]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

In [ ]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=256)
rnn_cnn_k_max2 = np.argmax(all_test_preds,axis=1)[:]+1
pd.Series(rnn_cnn_k_max2).value_counts(normalize=True)

In [ ]:
np.savetxt("rnn-cnn-0.6770-result.txt", rnn_cnn_k_max2,fmt="%d")

In [ ]:
all_test_preds = rnn_cnn_model.predict(padded_test_dataset_sequences, batch_size=256)
w2v6 = np.argmax(all_test_preds,axis=1)[:]+1

In [ ]:
pd.Series(w2v6).value_counts(normalize=True)

In [ ]:
np.savetxt("rnn-cnn-0.6320-result.txt", w2v6,fmt="%d")

In [ ]:
print('Build model...')
model = Sequential()
model.add(pre_embedding_layer)
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1, return_sequences=True)))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1)))
model.add(Dense(NUM_LABELS, activation='softmax'))

In [ ]:
def get_rnn_cnn_model():
    embedding_dim = 300
    inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    x =  Embedding(input_dim = nb_words+1, 
                             output_dim = EMBEDDING_DIM, 
                             weights=[word_embedding_matrix], 
                             input_length=MAX_SEQUENCE_LENGTH, 
                             mask_zero=False,
                             trainable=False
                            )(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
#     max_pool = GlobalMaxPooling1D()(x)
    kmax_pool = Lambda(lambda x: K.max(x, axis=1), output_shape=(64,))(x)
    conc = concatenate([avg_pool, kmax_pool])
    outp = Dense(3, activation="softmax")(conc)

    model = Model(inputs=inp, outputs=outp)
    
    model.compile(loss='categorical_crossentropy',
              #optimizer='adam',
                  optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
    
    return model

In [ ]:
rnn_cnn_model = get_rnn_cnn_model()

In [ ]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)

In [ ]:
batch_size = 256
epochs = 5

# model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
#                     validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
#                     batch_size=batch_size, 
#                     epochs=epochs,
#           shuffle=True,
#                     verbose=1
#          )
history = rnn_cnn_model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=3), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=3)[:]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    shuffle=True,
                    verbose=1)